In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib as plt
import os


In [2]:
#set working directory
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print("Working Directory: ", os.getcwd())

Working Directory:  n:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance


In [ ]:
labels = pd.read_csv('data/raw/train_labels.csv')

In [ ]:
labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

In [ ]:
#deleting the now obsolete labels of the labels
labels= labels.set_index('session')
labels = labels.drop(["session_id"], axis= 1)


In [3]:
#saving the dataset



labels.to_csv("data/processed/labels.csv",index_label= "session")

NameError: name 'labels' is not defined

The Dataframe labels is saved. 
it contains the cols:
 session : the session id
 correct : 0 and 1 for the answer of the question
 q: number of the question from 1 to 18

The Data are stored as a .csv  

to load the data efficiently use this loading function



In [4]:
dtypes_labels= {
    'correct': np.uint8, 
    'q':np.uint8}

labels = pd.read_csv('data/processed/labels_q1-3.csv', dtype=dtypes_labels)

In [5]:
def load_train_data(file_path: str , dtypes: dict = None, n_rows: int = None):
    # If dtypes is not specified, set default data types for each column
    if dtypes is None:
        dtypes = {
            'level': np.uint8,  
            'session_id': np.int64,
            'level_group': 'category',
            'event_name': np.uint8,
            'name': np.uint8,
            'fqid': np.uint8,
            'room_fqid': np.uint8,
            'text_fqid': np.uint8,
            'fullscreen': np.uint8,
            'hq': np.uint8,
            'music': np.uint8,
            'hover_duration_mean': np.float32,
            'difference_clicks_mean': np.float32,
            "distance_clicks_mean": np.float32,
            "screen_distance_clicks_mean": np.float32,            
            'elapsed_time_std': np.float32,
            'page_std': np.float32,
            'room_coor_x_std': np.float32,
            'room_coor_y_std': np.float32,
            'screen_coor_x_std': np.float32,
            'screen_coor_y_std': np.float32,
            'hover_duration_std': np.float32,
            'difference_clicks_std': np.float32,
            "distance_clicks_std": np.float32,
            "screen_distance_clicks_std": np.float32,
            'index_sum_of_actions': np.int32,
            'difference_clicks_max': np.float32,
            'elapsed_time_max': np.float32,
            'clicks_per_second': np.float32,
            "sum_distance_clicks_max": np.float32,
        }
        
    # Read in the CSV file
    if n_rows is None:
        df = pd.read_csv(file_path, dtype=dtypes, index_col = 0)
    else:
        df = pd.read_csv(file_path, dtype=dtypes, nrows=n_rows, index_col= 0)
    
    # Set data types for columns with "_i" index in their name
    row, cols = df.shape
    if cols > 50:
        for column in df.columns:
            base_name = column.rsplit('_', 1)[0]  # get the base name by splitting on the last "_" character
            if base_name in dtypes:
                column_number = column.rsplit('_', 1)[1]  # get the number from the index by splitting on the last "_" character
                new_column_name = f"{base_name}_{column_number}"  # construct the new column name
                column_dtype = dtypes[base_name]
                df[new_column_name] = df[column].astype(column_dtype)  # set the same data type for all columns with the same base name

    return df

In [6]:
train_data = load_train_data("data/processed/df_0_4_flattened.csv")

In [25]:
labels_q1 = labels[labels["q"] == 1]
labels_q1.rename(columns= {"session": "session_id_1"}, inplace= True)

C:\Users\steph\AppData\Local\Temp\ipykernel_9160\2564891973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_q1.rename(columns= {"session": "session_id_1"}, inplace= True)


In [26]:
merged_df = pd.merge(labels_q1, train_data, on='session_id_1')


In [27]:
y = merged_df["correct"]
X = merged_df.drop(columns=["session_id_1", "level_group_1", "q", "correct"])


In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import numpy as np

# Assuming you have a DataFrame of features called "df_X" (shape: [n_samples, 107]) and
# a target column called "df_y" (shape: [n_samples]).


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to oversample the minority class (0)
smote = SMOTE(sampling_strategy='minority')
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)



# Define the MLP architecture
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with class weights
epochs = 10
batch_size = 32
model.fit(X_train_resampled, y_train_resampled, epochs=epochs, batch_size=batch_size, class_weight=class_weights_dict)

# Evaluate the model on the testing set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary predictions

# Calculate evaluation metrics
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print('F1 Score:', f1)
print('Precision:', precision)
print('Recall:', recall)
print('Confusion Matrix:')
print(conf_matrix)


Epoch 1/10
856/856 [==============================] - 2s 2ms/step - loss: 39200359448576.0000 - accuracy: 0.4972
Epoch 2/10
856/856 [==============================] - 2s 2ms/step - loss: 69312839680.0000 - accuracy: 0.4967
Epoch 3/10
856/856 [==============================] - 2s 2ms/step - loss: 0.6932 - accuracy: 0.4939
Epoch 4/10
856/856 [==============================] - 2s 2ms/step - loss: 0.6932 - accuracy: 0.4933
Epoch 5/10
856/856 [==============================] - 2s 2ms/step - loss: 0.6932 - accuracy: 0.4930
Epoch 6/10
856/856 [==============================] - 2s 2ms/step - loss: 0.6932 - accuracy: 0.4954
Epoch 7/10
856/856 [==============================] - 2s 2ms/step - loss: 0.6932 - accuracy: 0.4949
Epoch 8/10
856/856 [==============================] - 2s 2ms/step - loss: 0.6932 - accuracy: 0.5014
Epoch 9/10
856/856 [==============================] - 2s 2ms/step - loss: 0.6932 - accuracy: 0.4985
Epoch 10/10
148/148 [==============================] - 0s 866us/step
F1 Score

In [28]:
unique, counts = np.unique(y_pred, return_counts=  True)
print(dict(zip(unique, counts)))

{0: 4, 1: 4709}


In [49]:
print(predictions)

[[ True]
 [ True]
 [ True]
 ...
 [ True]
 [ True]
 [ True]]


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Assuming you have a DataFrame of features called "X" (shape: [n_samples, 107]) and
# a target column called "df_y" (shape: [n_samples]).
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to oversample the minority class (0)
smote = SMOTE(sampling_strategy='minority')
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define the Random Forest classifier
model = RandomForestClassifier()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Create a grid search object
grid_search = GridSearchCV(model, param_grid=param_grid)

# Train the model using the grid search
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Evaluate the best model on the testing set
y_pred = best_model.predict(X_test)

# Calculate evaluation metrics
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print('F1 Score:', f1)
print('Precision:', precision)
print('Recall:', recall)
print('Confusion Matrix:')
print(conf_matrix)

F1 Score: 0.8287642926022869
Precision: 0.7913706919231781
Recall: 0.869866975130133
Confusion Matrix:
[[ 462  793]
 [ 450 3008]]


In [31]:
from joblib import dump, load

# Save the best model to a file
dump(best_model, 'models/steves_first_decisiontree_best_model_q1.joblib')

# Load the best model from a file
#best_model = load('models/steves_first_decisiontree_best_model.joblib')

['models/steves_first_decisiontree_best_model_q1.joblib']